In [1]:
%load_ext autoreload
%autoreload 2

# Getting Started Tutorial with Tabular ML Toolkit

> A tutorial on getting started with Tabular ml toolkit

> tabular_ml_toolkit is a superfast helper library to speedup your machine learning project based on Tabular or Structured data.

> It comes with model parallelism and cutting edge hyperparameter tuning techniques.

## Install

`pip install -U tabular_ml_toolkit`

## How to Best Use tabular_ml_toolkit

Start with your favorite model and then just simply create MLPipeline with one API.

*For example, Here we are using RandomForestRegressor from Scikit-Learn, on  [Melbourne Home Sale price data](https://www.kaggle.com/estrotococo/home-data-for-ml-course)*


*No need to install scikit-learn as it comes preinstall with Tabular_ML_Toolkit*

In [2]:
from tabular_ml_toolkit.mlpipeline import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np

# for displaying diagram of pipelines 
from sklearn import set_config
set_config(display="diagram")

# Just to compare fit times
import time

/Users/pankajmathur/anaconda3/envs/nbdev_env/lib/python3.9/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


In [3]:
# Dataset file names and Paths
DIRECTORY_PATH = "https://raw.githubusercontent.com/psmathur/tabular_ml_toolkit/master/input/home_data/"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"

In [4]:
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators':250,
    'learning_rate':0.05,
    'random_state':42,
    # for GPU
#     'tree_method': 'gpu_hist',
#     'predictor': 'gpu_predictor',
}


# create xgb ml model
xgb_model = XGBRegressor(**xgb_params)

In [5]:
# createm ml pipeline for scikit-learn model
tmlt = MLPipeline().prepare_data_for_training(
    train_file_path= DIRECTORY_PATH+TRAIN_FILE,
    test_file_path= DIRECTORY_PATH+TEST_FILE,
    idx_col="Id", target="SalePrice",
    model=xgb_model,
    random_state=42)

2021-11-15 23:27:44,001 INFO 8 cores found, parallel processing is enabled!
2021-11-15 23:27:44,292 INFO DataFrame Memory usage decreased to 0.58 Mb (35.5% reduction)
2021-11-15 23:27:44,861 INFO DataFrame Memory usage decreased to 0.58 Mb (34.8% reduction)


In [6]:
# tmlt.spl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Lo...
                              importance_type='gain',
                              interaction_constraints=None, learning_rate=0.05,
                              max_delta_step=None, max_depth=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=250,
                              n_jobs=7, num_parallel_tree=None, random_state=42,
                              reg_alpha=None, reg_lambda=None,
                              scale_pos_weight=None, subsample=None,
                              tree_method=None, validate_parameters=None,
                              verbosity=None))])

#### To see clear picture, let's do k_fold training on updated scikit model

In [11]:
# # k-fold training
# xgb_model_metrics_score, xgb_model_preds = tmlt.do_k_fold_training(n_splits=5,
#                                                                           metrics=mean_absolute_error,
#                                                                           random_state=42)
# print("mean metrics score:", np.mean(xgb_model_metrics_score))
# # predict
# print(xgb_model_preds.shape)

/Users/pankajmathur/anaconda3/envs/nbdev_env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2021-11-15 23:27:51,400 INFO fold: 1 , mean_absolute_error: 18947.19236943493
2021-11-15 23:27:52,005 INFO fold: 2 , mean_absolute_error: 15652.96465646404
2021-11-15 23:27:52,662 INFO fold: 3 , mean_absolute_error: 16128.323335830479
2021-11-15 23:27:53,367 INFO fold: 4 , mean_absolute_error: 15037.816045055652
2021-11-15 23:27:54,048 INFO fold: 5 , mean_absolute_error: 17555.253585188355


mean metrics score: 16664.309998394692
(1459,)


##### Let's see if we can improve our K_Fold score with hyperparams tunning

In [19]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.impute import SimpleImputer
# # from sklearn.

In [20]:
study = tmlt.do_xgb_optuna_optimization(task="regression", xgb_eval_metric="rmse",
                                        kfold_metrics=mean_absolute_error, output_dir_path="output/")
print(study.best_trial)

2021-11-15 23:45:44,713 INFO direction is: minimize
[I 2021-11-15 23:45:44,735] Using an existing study with name 'tmlt_autoxgb' instead of creating a new one.
/Users/pankajmathur/anaconda3/envs/nbdev_env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[23:45:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:46:27,876 INFO fold: 1 , mean_absolute_error: 18639.659139554795


[23:46:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:47:15,923 INFO fold: 2 , mean_absolute_error: 14579.810439854453


...

[23:52:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:53:19,235 INFO fold: 4 , mean_absolute_error: 14190.637146832192


[23:53:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:53:43,141 INFO fold: 5 , mean_absolute_error: 15427.863762842466
[I 2021-11-15 23:53:43,160] Trial 35 finished with value: 15422.884051262843 and parameters: {'learning_rate': 0.017176534972555314, 'reg_lambda': 6.379202083079888e-07, 'reg_alpha': 0.013147773254711077, 'subsample': 0.5559462698979399, 'colsample_bytree': 0.5526981282005409, 'max_depth': 8, 'early_stopping_rounds': 229, 'n_estimators': 7000, 'tree_method': 'hist', 'booster': 'gbtree', 'gamma': 0.00010121273930113025, 'grow_policy': 'depthwise'}. Best is trial 31 with value: 15161.862516053083.


FrozenTrial(number=31, values=[15161.862516053083], datetime_start=datetime.datetime(2021, 11, 15, 23, 34, 33, 726431), datetime_complete=datetime.datetime(2021, 11, 15, 23, 35, 51, 401636), params={'booster': 'gbtree', 'colsample_bytree': 0.7978993899268726, 'early_stopping_rounds': 219, 'gamma': 0.00012602511463355868, 'grow_policy': 'depthwise', 'learning_rate': 0.015379020226564632, 'max_depth': 5, 'n_estimators': 7000, 'reg_alpha': 0.03253148458263187, 'reg_lambda': 3.208442870212238e-07, 'subsample': 0.6347778191052607, 'tree_method': 'hist'}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear')), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'early_stopping_rounds': IntUniformDistribution(high=500, low=100, step=1), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide')), 'learning_rate': LogUniformDistribution(high=0.25, low=0.01), 'max_depth': IntUniformDistributi

In [21]:
study.best_trial.params

{'booster': 'gbtree',
 'colsample_bytree': 0.7978993899268726,
 'early_stopping_rounds': 219,
 'gamma': 0.00012602511463355868,
 'grow_policy': 'depthwise',
 'learning_rate': 0.015379020226564632,
 'max_depth': 5,
 'n_estimators': 7000,
 'reg_alpha': 0.03253148458263187,
 'reg_lambda': 3.208442870212238e-07,
 'subsample': 0.6347778191052607,
 'tree_method': 'hist'}

**Awesome we found best params with K-fold variations ~ 1 minute!**

##### Now let's use best params to update preprocessor and model in our pipeline

In [22]:
autoxgb_params = {'learning_rate': 0.016067642810265004,

'reg_lambda': 0.0005033307729410949,

'reg_alpha': 1.125131255655592e-06,

'subsample': 0.43211847297916883,

'colsample_bytree': 0.4106787563173376,

'max_depth': 5,

'early_stopping_rounds': 354,

'n_estimators': 7000,

'tree_method': 'approx',

'booster': 'gbtree',

'gamma': 0.2870988185671683,

'grow_policy': 'depthwise'}

In [23]:
# xgb_params = autoxgb_params
tmlt_xgb_params = study.best_trial.params
xgb_params =  tmlt_xgb_params
xgb_model = XGBRegressor(**xgb_params)
tmlt.update_model(xgb_model)
tmlt.spl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Lo...
                              learning_rate=0.015379020226564632,
                              max_delta_step=None, max_depth=5,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=7000,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, reg_alpha=0.03253148458263187,
                              reg_lambda=3.208442870212238e-07,
                              scale_pos_weight=None,
                              subsample=0.6347778191052607, tree_method='hist',
                              validate_parameters=None, verbosity=None))])

In [18]:
# k-fold training
xgb_model_metrics_score, xgb_model_preds = tmlt.do_k_fold_training(n_splits=5,
                                                                          metrics=mean_absolute_error,
                                                                          random_state=42)
print("mean metrics score:", np.mean(xgb_model_metrics_score))
# predict
print(xgb_model_preds.shape)

/Users/pankajmathur/anaconda3/envs/nbdev_env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[23:44:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:44:19,642 INFO fold: 1 , mean_absolute_error: 18305.722629494863


[23:44:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:44:33,580 INFO fold: 2 , mean_absolute_error: 14528.402704944348


...

[23:44:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:45:02,359 INFO fold: 4 , mean_absolute_error: 13539.36665239726


[23:45:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021-11-15 23:45:16,811 INFO fold: 5 , mean_absolute_error: 15188.842867080479


mean metrics score: 15085.97251177226
(1459,)


mean metrics score: 15447.19800406678
(1459,)

#### Yup Indeed Optuna tunning xgb model has improved MAE from earlier cross validated model!

**Amazing our MAE has reduced to 15689.22 by GridSearch HyperParamss tunning, If we can continue doing hyperparmas tunning, may be we can even do better, take that as challenge!**